In [7]:
import os
import shutil
import opendatasets as od

In [37]:
dataset_url = ("https://www.kaggle.com/datasets/rtatman/british-birdsong-dataset?resource=download")
dowload_path = "..\\datasets\\"

od.download(dataset_url, data_dir=dowload_path)

dataset_path = dowload_path + "british-birdsong-dataset\\"

100%|██████████| 633M/633M [00:23<00:00, 28.8MB/s] 


In [38]:
# Remove redundant subfolder (only for british birdsong dataset)
old_path = dataset_path + 'songs\\songs\\'
audio_path = dataset_path + 'audio\\'

os.mkdir(audio_path)

for file in os.listdir(old_path):
    os.replace(old_path + file, audio_path + file)

shutil.rmtree(dataset_path + '\\songs')